In [1]:
import tsnet
import numpy as np
import matplotlib.pyplot as plt
import wntr

In [2]:
np.random.seed(1234)

In [3]:
# Open an example network and create a transient model
inp_file='../networks/net2.inp'
tm = tsnet.network.TransientModel(inp_file)
wn = wntr.network.WaterNetworkModel(inp_file)

In [4]:
# tm.set_wavespeed(1000.) # m/s
wavespeed = np.random.normal(1000., 100., size=tm.num_pipes)
tm.set_wavespeed(wavespeed)
# # Set time options
dt = 0.01
tf = 10   # simulation period [s]
tm.set_time(tf,dt)
#tm.set_time(tf)

Simulation time step 0.01136 s


In [5]:
# MOC 分段
MOC_n=tsnet.network.discretize.cal_N(tm,dt)
MOC_n[1]

array([5.])

In [6]:
ts = 0 # burst start time
tc = 0 # time for burst to fully develop
final_burst_coeff = 0.01 # final burst coeff [ m^3/s/(m H20)^(1/2)]
tm.add_burst('J00010', ts, tc, final_burst_coeff)

In [7]:
# Initialize steady state simulation
t0=0 # start time
tm = tsnet.simulation.Initializer(tm,t0,'DD')

Total Time Step in this simulation 879


In [8]:
# Transient simulation
results_obj = 'demo2_burst' # name of the object for saving simulation results
friction = 'unsteady' # "steady" or "unsteady" or "quasi-steady", by default "steady"
tm = tsnet.simulation.MOCSimulator(tm,results_obj,friction)

Initial condition discrepancy of pressure (-15.4529 m) on the J00010 node
Initial condition discrepancy of pressure (-15.4529 m) on the J00010 node
Estimated simulation time 0:00:02.051586
Transient simulation completed 9 %...
Transient simulation completed 19 %...
Transient simulation completed 29 %...
Transient simulation completed 39 %...
Transient simulation completed 49 %...
Transient simulation completed 59 %...
Transient simulation completed 69 %...
Transient simulation completed 79 %...
Transient simulation completed 89 %...
Transient simulation completed 98 %...


In [9]:
node = ['J00000','J00001','J00010','J00019']
tm.plot_node_head(node)

In [10]:
flow10=tm.get_link('P00010').start_node_flowrate

flow20=tm.get_link('P00018').start_node_flowrate
tt = tm.simulation_timestamps
plt.plot(tt,flow10)

plt.plot(tt,flow20)

# create the database

In [11]:
num_nodes=wn.num_nodes # col
row=len(tt)
flows=np.zeros([row,num_nodes])
heads=np.zeros([row,num_nodes])
flows.shape

(879, 23)

In [12]:
# 00001-J00019-J00020 and R00001
i=0
for pipe_name, pipe in wn.pipes():
    flow_tmp=tm.get_link(pipe_name).start_node_flowrate
    print(i,pipe_name)
    flows[:,i]=flow_tmp
    
    if i==9:
        i+=1
        flows[:,i]=tm.get_link(pipe_name).end_node_flowrate
        
    # if i==10: # PIPE-10-start
    #     
    #     flows[:,i]=tm.get_link(pipe_name).end_node_flowrate
    #     i+=1
    #     flows[:,i]=tm.get_link(pipe_name).start_node_flowrate
    #     # pipe10-start remember again
    
    if i==20:
        i+=1
        flows[:,i]=tm.get_link(pipe_name).end_node_flowrate
        # end node
    i+=1

0 P00000
1 P00001
2 P00002
3 P00003
4 P00004
5 P00005
6 P00006
7 P00007
8 P00008
9 P00009
11 P00010
12 P00011
13 P00012
14 P00013
15 P00014
16 P00015
17 P00016
18 P00017
19 P00018
20 P00019


In [13]:
flows[:,10]-flows[:,11]

array([0.35 , 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.536, 0.536, 0.535, 0.536, 0.535, 0.536, 0.535, 0.536, 0.537, 0.537, 0.537, 0.538, 0.537, 0.538, 0.537, 0.538, 0.537, 0.538, 0.536, 0.536, 0.536, 0.536, 0.536, 0.536, 0.536, 0.536, 0.536, 0.536,
       0.536, 0.536, 0.536, 0.536, 0.535, 0.536, 0.536, 0.537, 0.537, 0.537, 0.537, 0.537, 0.537, 0.537, 0.537, 0.538, 0.537, 0.538, 0.537, 0.538, 0.537, 0.538, 0.537, 0.538, 0.537, 0.538, 0.536, 0.536, 0.536, 0.536, 0.535, 0.536, 0.535, 0.535, 0.536, 0.536, 0.537, 0.537, 0.537, 0.537, 0.537,
       0.537, 0.537, 0.537, 0.536, 0.536, 0.536, 0.536, 0.535, 0.534, 0.533, 0.534, 0.534, 0.535, 0.538, 0.538, 0.539, 0.539, 0.539, 0.539, 0.539, 0.538, 0.538, 0.539, 0.54 , 0.54 , 0.542, 0.542, 0.543, 0.543, 0.543, 0.543, 0.542, 0.541, 0.54 , 0.54 , 0.54 , 0.54 , 0.54 , 0.539, 0.539, 0.539,
       0.539, 0.54 , 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.539, 0.54 , 0.541, 0.541, 0.542,

In [14]:
j=0
for node_name in wn.node_name_list:
    head_tmp=tm.get_node(node_name).head
    print(j,node_name)
    if j<num_nodes:
        heads[:,j]=head_tmp
    if j==10:
        j+=1
        heads[:,j]=head_tmp
    j+=1

0 J00000
1 J00001
2 J00002
3 J00003
4 J00004
5 J00005
6 J00006
7 J00007
8 J00008
9 J00009
10 J00010
12 J00011
13 J00012
14 J00013
15 J00014
16 J00015
17 J00016
18 J00017
19 J00018
20 J00019
21 J00020
22 J00021
23 R00001


In [15]:
heads[:,10]-heads[:,11]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [16]:
# t-head ; t-flow
datah=np.zeros([row,num_nodes+1])
dataq=np.zeros([row,num_nodes+1])
datah[:,0]=np.asarray(tt)
dataq[:,0]=np.asarray(tt)
datah[:,1:]=heads
dataq[:,1:]=flows

In [17]:
dataq

array([[0.   , 0.5  , 0.5  , ..., 0.15 , 0.15 , 0.   ],
       [0.011, 0.5  , 0.5  , ..., 0.15 , 0.15 , 0.   ],
       [0.023, 0.5  , 0.5  , ..., 0.15 , 0.15 , 0.   ],
       ...,
       [9.955, 0.678, 0.68 , ..., 0.149, 0.15 , 0.   ],
       [9.967, 0.68 , 0.682, ..., 0.149, 0.15 , 0.   ],
       [9.978, 0.682, 0.684, ..., 0.149, 0.15 , 0.   ]])

In [18]:
# save to csv
np.savetxt('demo2_burst_h.csv',datah,fmt='%.4f',delimiter=',')
np.savetxt('demo2_burst_q.csv',dataq,fmt='%.4f',delimiter=',')